In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
from random import randint
import pickle

In [43]:
browser = webdriver.Firefox()

In [44]:
browser.get('https://siftery.com')

login

In [4]:
browser.get('https://siftery.com/groups/ecommerce-1?activetab=products_used')

In [5]:
soup = BeautifulSoup(browser.page_source)

/usr/local/lib/python2.7/dist-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [6]:
divs = soup.find('div',{'class':'tab-content'}).find_all('div',{'class':'products-bucket ng-scope'})

In [7]:
divs[0].find('h3').text

u'Marketing'

In [8]:
divs[0].find_all('div',{'class':'cell-item cell-group-item cell-item--small ng-scope'})[0].find('a')['href']

'/google-tag-manager?group=ecommerce-1'

In [9]:
from url_cleaner import UrlCleaner
urlcleaner = UrlCleaner()

In [10]:
urlcleaner.merge_urls('https://siftery.com',
        divs[0].find_all('div',{'class':'cell-item cell-group-item cell-item--small ng-scope'})[0].find('a')['href'])

'https://siftery.com/google-tag-manager?group=ecommerce-1'

In [26]:
browser.get('https://siftery.com/g-suite-formerly-google-apps-for-work?group=ecommerce-1'+'&page={}'.format(2))
i=0
while i<11:
    time.sleep(2)
    soup = BeautifulSoup(browser.page_source)
    if soup.find('div',{'class':'cell-container ng-scope'}):
        break
    else:
        i += 1

In [29]:
html == html_1

False

In [30]:
html = html_1
html_1 = browser.page_source
soup = BeautifulSoup(html)

In [31]:
soup1 = BeautifulSoup(html_1)

In [32]:
soup.find('div',{'class':'cell-container ng-scope'})

In [39]:
soup1.find('div',{'class':'cell-container ng-scope'}).\
    find_all('div',{'class':'cell-item cell-item--medium ng-scope'})[0].find('a')['href']

'/company/wanelo'

In [68]:
soup1.find('div',{'class':'cell-container ng-scope'}).\
    find_all('div',{'class':'cell-item cell-item--medium ng-scope'})[0].find('a').text

u'Wanelo'

In [81]:
soup1.find('div',{'class':'sidebar-body'}).find('section',{'class':'sidebar-section ng-scope'}).text

u'G Suite (formerly Google Apps for Work)Google Apps for Work gives team members professional email services along with online storage, shared calendars, video  and more.'

In [85]:
[i.text for i in soup1.find('div',{'class':'sidebar-body'}).find_all('section',{'class':'sidebar-section ng-scope'})[1].\
            find_all('li',{'class':'ng-scope'})]

[u' Hosted Email', u' Office Suites']

In [41]:
urls = [urlcleaner.merge_urls('https://siftery.com',i.find('a')['href'])
                 for i in divs[0].find_all('div',{'class':'cell-item cell-group-item cell-item--small ng-scope'})]

In [43]:
urls[0]

'https://siftery.com/g-suite-formerly-google-apps-for-work?group=ecommerce-1'

In [64]:
tech_urls = {}
for div in divs:
    urls = [(urlcleaner.merge_urls('https://siftery.com',i.find('a')['href']),i.find('h4').text)
            for i in div.find_all('div',{'class':'cell-item cell-group-item cell-item--small ng-scope'})]
    tech = div.find('h3').text
    tech_urls[tech] = urls

In [63]:
div.find_all('div',{'class':'cell-item cell-group-item cell-item--small ng-scope'})[0].find('h4').text

u'Hive'

In [65]:
tech_urls_1 = {'tech_class':[],'tech_name':[],'tech_siftery_url':[]}
for key in tech_urls.keys():
    for i in tech_urls[key]:
        tech_urls_1['tech_class'].append(key)
        tech_urls_1['tech_name'].append(i[1])
        tech_urls_1['tech_siftery_url'].append(i[0])
import pandas as pd
tech_urls_df = pd.DataFrame(tech_urls_1)
tech_urls_df.head()

,tech_class,tech_name,tech_siftery_url
0,Customer Support and Success,Zendesk,https://siftery.com/zendesk?group=ecommerce-1
1,Customer Support and Success,Zopim,https://siftery.com/zopim?group=ecommerce-1
2,Customer Support and Success,ForeSee,https://siftery.com/foresee?group=ecommerce-1
3,Customer Support and Success,OpinionLab,https://siftery.com/opinionlab?group=ecommerce-1
4,Customer Support and Success,Salesforce Desk.com,https://siftery.com/salesforce-deskcom?group=e...


In [67]:
tech_urls_df.to_excel('siftery_data/technologies.xls',index=False)

In [95]:
tech_comps_dic = []

In [90]:
import pickle
from random import shuffle

In [96]:
tech_urls_list = tech_urls_df['tech_siftery_url']
for url in tech_urls_list:
    print url
    for ind in range(5)[1:]:
        try:
            browser.get(url+'&page={}'.format(ind))
            i=0
            while i<11:
                time.sleep(2)
                soup1 = BeautifulSoup(browser.page_source)
                if soup1.find('div',{'class':'cell-container ng-scope'}):
                    break
                else:
                    i += 1
            description = soup1.find('div',{'class':'sidebar-body'}).\
                    find('section',{'class':'sidebar-section ng-scope'}).text
            categories = [i.text for i in soup1.find('div',{'class':'sidebar-body'}).\
                          find_all('section',{'class':'sidebar-section ng-scope'})[1].\
                            find_all('li',{'class':'ng-scope'})]
            divs = soup1.find('div',{'class':'cell-container ng-scope'}).\
                find_all('div',{'class':'cell-item cell-item--medium ng-scope'})
            for div in divs:
                tech_comps_dic.append((url,div.find('a').text,div.find('a')['href'],description,categories))
            time.sleep(2)
            with open('siftery_data/techs_comps.pkl','w') as f:
                pickle.dump(tech_comps_dic,f)
        except:
            continue

https://siftery.com/zendesk?group=ecommerce-1
https://siftery.com/zopim?group=ecommerce-1
https://siftery.com/foresee?group=ecommerce-1
https://siftery.com/opinionlab?group=ecommerce-1
https://siftery.com/salesforce-deskcom?group=ecommerce-1
https://siftery.com/olark?group=ecommerce-1
https://siftery.com/bazaarvoice?group=ecommerce-1
https://siftery.com/salesforce-service-cloud?group=ecommerce-1
https://siftery.com/clarabridge-cx-suite?group=ecommerce-1
https://siftery.com/qualaroo?group=ecommerce-1
https://siftery.com/uservoice?group=ecommerce-1
https://siftery.com/qualtrics-research-suite?group=ecommerce-1
https://siftery.com/livechat?group=ecommerce-1
https://siftery.com/delighted?group=ecommerce-1
https://siftery.com/wufoo?group=ecommerce-1
https://siftery.com/liveperson?group=ecommerce-1
https://siftery.com/fuel-cycle-by-passenger?group=ecommerce-1
https://siftery.com/intercom?group=ecommerce-1
https://siftery.com/nanorep?group=ecommerce-1
https://siftery.com/cx-company?group=ecom

In [100]:
browser.quit()

In [99]:
tmp1 = pd.DataFrame.from_records(tech_comps_dic)
tmp1.columns = ['siftery_tech_url','company_name','siftery_company_url','tech_description','tech_categories']
tmp1.to_csv('siftery_data/techs_comps.csv',index=False,encoding='utf-8')

now crawl all the companies

In [6]:
import pandas as pd

In [7]:
comp_df = pd.read_excel('siftery_data/techs_comps.xls')

In [8]:
comp_df.head()

,siftery_tech_url,company_name,siftery_company_url,tech_description,tech_categories
0,https://siftery.com/zendesk?group=ecommerce-1,Raise,/company/raise,ZendeskZendesk is a software development firm ...,[u' Help Desk and Ticketing']
1,https://siftery.com/zendesk?group=ecommerce-1,Everything But The House,/company/everything-but-the-house,ZendeskZendesk is a software development firm ...,[u' Help Desk and Ticketing']
2,https://siftery.com/zendesk?group=ecommerce-1,Everlane,/company/everlane,ZendeskZendesk is a software development firm ...,[u' Help Desk and Ticketing']
3,https://siftery.com/zendesk?group=ecommerce-1,Dollar Shave Club,/company/dollar-shave-club,ZendeskZendesk is a software development firm ...,[u' Help Desk and Ticketing']
4,https://siftery.com/zendesk?group=ecommerce-1,Blue Apron,/company/blue-apron,ZendeskZendesk is a software development firm ...,[u' Help Desk and Ticketing']


In [9]:
comps = list(set(comp_df['siftery_company_url']))
comps = ['https://siftery.com'+i for i in comps]
comps[:2],len(comps)

([u'https://siftery.com/company/social-point',
  u'https://siftery.com/company/florida-today'],
 6145)

In [10]:
browser.get(comps[0])

In [12]:
soup = BeautifulSoup(browser.page_source)

In [48]:
soup.find('h1',{'class':'header-item__title'}).find('strong').text

u'Social Point'

In [54]:
soup.find('h4').find('a')['href']

'http://www.socialpoint.es'

In [41]:
cat_div = soup.find('div',{'class':'fluid-section-body__inner'}).find_all('div',{'class':'products-bucket ng-scope'})[1]

In [42]:
cat_div.find('h3').text

u'Customer Support and Success'

In [43]:
len(cat_div.find_all('div',{'class':'cell-item cell-item--large ng-scope'}))

3

In [44]:
prod_div = cat_div.find_all('div',{'class':'cell-item cell-item--large ng-scope'})[1]

In [45]:
prod_div.find('a',{'class':'block'}).find('h4').text.strip()

u'Polljoy'

In [46]:
prod_div.find('a',{'class':'block'})['href']

'/polljoy'

In [47]:
prod_div.find('a',{'class':'block'}).find('p').text

u'A prompt for feedback and 5 star ratings on Unity, iOS, Android and HTML5'

In [12]:
comp_dets_list = []
for url in comps[2583:]:
    try:
        browser.get(url)
        time.sleep(randint(5,15))
        soup = BeautifulSoup(browser.page_source)
        company_name = soup.find('h1',{'class':'header-item__title'}).find('strong').text
        try:
            company_website = soup.find('h4').find('a')['href']
        except:
            company_website = ''
        for cat_div in \
           soup.find('div',{'class':'fluid-section-body__inner'}).find_all('div',{'class':'products-bucket ng-scope'}):
            category = cat_div.find('h3').text
            category_prods_list = []
            for prod_div in cat_div.find_all('div',{'class':'cell-item cell-item--large ng-scope'}):
                prod_name = prod_div.find('a',{'class':'block'}).find('h4').text.strip()
                prod_siftery_url = prod_div.find('a',{'class':'block'})['href']
                prod_descr = prod_div.find('a',{'class':'block'}).find('p').text
                category_prods_list.append((prod_name,prod_descr,prod_siftery_url))
            comp_dets_list.append((url,company_name,company_website,category,category_prods_list))
            with open('siftery_data/companies_and_technologies1.pkl','w') as f:
                pickle.dump(comp_dets_list,f)
    except Exception as e:
        print('error happened for url: {}, exception: {}'.format(url,e))
        continue

/usr/local/lib/python2.7/dist-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


error happened for url: https://siftery.com/company/pipecandy, exception: 'NoneType' object has no attribute 'find'


In [13]:
comp_dets_df = pd.DataFrame.from_records(comp_dets_list)
comp_dets_df.columns = ['company_siftery_url','company_name','company_website','category','products_used']
comp_dets_df.to_excel('siftery_data/companies_and_technologies1.xls',index=False)

In [11]:
comps[2583]

u'https://siftery.com/company/uproxx'

crawling for specific categories for Sindhu

In [45]:
urls = ['https://siftery.com/categories/email-marketing/email-marketing-platforms',
       'https://siftery.com/categories/email-marketing/email-marketing-automation',
       'https://siftery.com/categories/email-marketing/email-tracking',
       'https://siftery.com/categories/email-marketing/email-personalization',
       'https://siftery.com/categories/email-marketing/drip-marketing',
       'https://siftery.com/categories/email-marketing/email-tracking',
       'https://siftery.com/categories/email-marketing/email-deliverability',
       'https://siftery.com/categories/email-marketing/email-personalization',
       'https://siftery.com/categories/email-marketing/email-analytics']

In [11]:
url = 'https://siftery.com/categories/email-marketing/email-marketing-platforms'
browser.get(url)

In [23]:
soup = BeautifulSoup(browser.page_source)

In [16]:
soup.find('div',{'class':'cell-container products-container'}).find_all('div',{'class':'cell-item ng-scope'})[0].\
    find('figcaption',{'class':'product-badge__caption'}).find('h4').text

u'Zoho Campaigns'

In [17]:
soup.find('div',{'class':'cell-container products-container'}).find_all('div',{'class':'cell-item ng-scope'})[0].\
    find('figcaption',{'class':'product-badge__caption'}).find('p').text

u'Zoho Campaigns is an email marketing tool with CRM & social media integration to help you create, send and track effective email campaigns.'

In [21]:
urlcleaner.merge_urls('https://siftery.com',soup.find('div',{'class':'cell-container products-container'}).find_all('div',{'class':'cell-item ng-scope'})[0].\
    find('div',{'class':'cell-entity'}).find('a')['href'])

'https://siftery.com/zoho-campaigns'

In [24]:
soup.find('div',{'class':'cell-container products-container'}).find_all('div',{'class':'cell-item ng-scope'})[-1].\
    find('figcaption',{'class':'product-badge__caption'}).find('h4').text

u'Vocus'

In [46]:
out_list = []
import pdb
for url in urls:
    browser.get(url)
    pdb.set_trace()
    soup = BeautifulSoup(browser.page_source)
    for div in soup.find('div',{'class':'cell-container products-container'}).\
        find_all('div',{'class':'cell-item ng-scope'}):
        name = div.find('figcaption',{'class':'product-badge__caption'}).find('h4').text
        text = div.find('figcaption',{'class':'product-badge__caption'}).find('p').text
        siftery_url = div.find('div',{'class':'cell-entity'}).find('a')['href']
        out_list.append((url,name,text,siftery_url))

> <ipython-input-46-7abdef48c2d2>(6)<module>()
-> soup = BeautifulSoup(browser.page_source)
(Pdb) c
> <ipython-input-46-7abdef48c2d2>(5)<module>()
-> pdb.set_trace()
(Pdb) len(out_list)
70
(Pdb) c
> <ipython-input-46-7abdef48c2d2>(6)<module>()
-> soup = BeautifulSoup(browser.page_source)
(Pdb) c
> <ipython-input-46-7abdef48c2d2>(5)<module>()
-> pdb.set_trace()
(Pdb) c
> <ipython-input-46-7abdef48c2d2>(6)<module>()
-> soup = BeautifulSoup(browser.page_source)
(Pdb) c
> <ipython-input-46-7abdef48c2d2>(5)<module>()
-> pdb.set_trace()
(Pdb) c
> <ipython-input-46-7abdef48c2d2>(6)<module>()
-> soup = BeautifulSoup(browser.page_source)
(Pdb) c
> <ipython-input-46-7abdef48c2d2>(5)<module>()
-> pdb.set_trace()
(Pdb) c
> <ipython-input-46-7abdef48c2d2>(6)<module>()
-> soup = BeautifulSoup(browser.page_source)
(Pdb) c


In [47]:
import pandas as pd

In [48]:
tmp = pd.DataFrame.from_records(out_list)
tmp.columns = ['siftery_url_category','tool_name','tool_description','tool_siftery_url']
tmp.head()

,siftery_url_category,tool_name,tool_description,tool_siftery_url
0,https://siftery.com/categories/email-marketing...,Zoho Campaigns,Zoho Campaigns is an email marketing tool with...,/zoho-campaigns
1,https://siftery.com/categories/email-marketing...,Net-Results,Net-Results helps companies increase revenue g...,/net-results
2,https://siftery.com/categories/email-marketing...,Mailjet,Mailjet is a real-time cloud emailing platform...,/mailjet
3,https://siftery.com/categories/email-marketing...,Mailee,Mailee is a tool to create and send newsletter...,/mailee
4,https://siftery.com/categories/email-marketing...,Benchmark,Benchmark provides online marketing tools to k...,/benchmark


In [58]:
tmp['tool_siftery_url'] = [urlcleaner.merge_urls('https://siftery.com',i) for i in tmp['tool_siftery_url']]
tmp.head()

,siftery_url_category,tool_name,tool_description,tool_siftery_url
0,https://siftery.com/categories/email-marketing...,Zoho Campaigns,Zoho Campaigns is an email marketing tool with...,https://siftery.com/zoho-campaigns
1,https://siftery.com/categories/email-marketing...,Net-Results,Net-Results helps companies increase revenue g...,https://siftery.com/net-results
2,https://siftery.com/categories/email-marketing...,Mailjet,Mailjet is a real-time cloud emailing platform...,https://siftery.com/mailjet
3,https://siftery.com/categories/email-marketing...,Mailee,Mailee is a tool to create and send newsletter...,https://siftery.com/mailee
4,https://siftery.com/categories/email-marketing...,Benchmark,Benchmark provides online marketing tools to k...,https://siftery.com/benchmark


In [63]:
tmp1 = tmp.groupby('tool_siftery_url').agg(lambda x: ','.join(x))

In [65]:
tmp.to_excel('siftery_data/email_marketing_tools.xls',index=False)
tmp1.to_excel('siftery_data/email_marketing_tools_grouped.xls')

Finding companies using these technologies

In [60]:
urls = list(set(tmp['tool_siftery_url']))

In [67]:
len(urls),tmp.shape,tmp1.shape

(209, (275, 4), (209, 3))

In [69]:
tmp1['tool_siftery_url'] = tmp1.index

In [70]:
tmp1.head()

,siftery_url_category,tool_name,tool_description,tool_siftery_url
tool_siftery_url,,,,
https://siftery.com/250ok,https://siftery.com/categories/email-marketing...,250ok,250ok offers a SaaS-based platform to monitor ...,https://siftery.com/250ok
https://siftery.com/360-campaigner,https://siftery.com/categories/email-marketing...,360 Campaigner,B2B email marketing automation platform with e...,https://siftery.com/360-campaigner
https://siftery.com/aboardly,https://siftery.com/categories/email-marketing...,Aboardly,Aboardly helps you send emails to your custome...,https://siftery.com/aboardly
https://siftery.com/act-on,https://siftery.com/categories/email-marketing...,Act-On,Act-On provides cloud-based integrated marketi...,https://siftery.com/act-on
https://siftery.com/act-premium,https://siftery.com/categories/email-marketing...,Act! Premium,Act! Premium helps organize all your prospect ...,https://siftery.com/act-premium


In [72]:
urls = list(tmp1.index)
urls[:4]

['https://siftery.com/250ok',
 'https://siftery.com/360-campaigner',
 'https://siftery.com/aboardly',
 'https://siftery.com/act-on']

In [88]:
tech_comps_dic = []
import re

In [89]:
tech_urls_list = urls
for url in tech_urls_list:
    print url
    for ind in range(5)[1:]:
        try:
            browser.get(url+'?page={}'.format(ind))
            i=0
            while i<11:
                time.sleep(2)
                soup1 = BeautifulSoup(browser.page_source)
                if soup1.find('div',{'class':'cell-container ng-scope'}):
                    break
                else:
                    i += 1
#             description = soup1.find('div',{'class':'sidebar-body'}).\
#                     find('section',{'class':'sidebar-section ng-scope'}).text
            categories = [i.text for i in soup1.find('div',{'class':'sidebar-body'}).\
                          find_all('section',{'class':'sidebar-section ng-scope'})[1].\
                            find_all('li',{'class':'ng-scope'})]
            divs = soup1.find('div',{'class':'cell-container ng-scope'}).\
                find_all('div',{'class':'cell-item cell-item--medium ng-scope'})
            for div in divs:
                tech_comps_dic.append((url,div.find('a').text,
                                       urlcleaner.merge_urls('https://siftery.com',div.find('a')['href']),categories))
            time.sleep(2)
            with open('siftery_data/techs_comps_email_related.pkl','w') as f:
                pickle.dump(tech_comps_dic,f)
            if re.search('Uh Oh! The page you were looking for doesn’t exist',soup1.text) or \
                re.search("have any customers to show for this product",soup1.text):
                break
            if len(divs)<25:
                break
        except:
            if re.search('Uh Oh! The page you were looking for doesn’t exist',soup1.text) or \
                re.search("have any customers to show for this product",soup1.text):
                break
            continue

https://siftery.com/250ok
https://siftery.com/360-campaigner
https://siftery.com/aboardly
https://siftery.com/act-on
https://siftery.com/act-premium
https://siftery.com/activecampaign
https://siftery.com/adestra
https://siftery.com/adobe-business-catalyst
https://siftery.com/appmail
https://siftery.com/apsis
https://siftery.com/aurea-email-marketing-lyris
https://siftery.com/authsmtp
https://siftery.com/automizy
https://siftery.com/autopilot
https://siftery.com/aweber
https://siftery.com/bananatag
https://siftery.com/benchmark
https://siftery.com/bluecore
https://siftery.com/bluehornet
https://siftery.com/blueshift
https://siftery.com/bombbomb
https://siftery.com/boomerang-for-gmail
https://siftery.com/boomerang-for-outlook
https://siftery.com/bounceio
https://siftery.com/briteverify
https://siftery.com/bronto
https://siftery.com/byside
https://siftery.com/campaign-monitor
https://siftery.com/campaigner
https://siftery.com/cirrus-insight
https://siftery.com/clang
https://siftery.com/cl

In [81]:
print tech_comps_dic[-1]

('https://siftery.com/250ok', u'Copernica', 'https://siftery.com/company/copernica', [u' Email Deliverability'])


In [90]:
tmpdf = pd.DataFrame.from_records(tech_comps_dic)

In [93]:
tmpdf.head()

,tech_siftery_url,company_name,company_siftery_url,tech_categories
0,https://siftery.com/250ok,Adobe,https://siftery.com/company/adobe,[ Email Deliverability]
1,https://siftery.com/250ok,Marketo,https://siftery.com/company/marketo,[ Email Deliverability]
2,https://siftery.com/250ok,Act-On Software,https://siftery.com/company/act-on-software,[ Email Deliverability]
3,https://siftery.com/250ok,DK New Media,https://siftery.com/company/dk-new-media,[ Email Deliverability]
4,https://siftery.com/250ok,Salesfusion,https://siftery.com/company/salesfusion,[ Email Deliverability]


In [91]:
tmpdf.columns = ['tech_siftery_url','company_name','company_siftery_url','tech_categories']

In [92]:
tmpdf.to_excel('siftery_data/email_tools_techs_comps.xls',index=False)

In [94]:
tmpdf.groupby('tech_siftery_url').agg(lambda x: ','.join(x)).head()

,company_name,company_siftery_url
tech_siftery_url,,
https://siftery.com/250ok,"Adobe,Marketo,Act-On Software,DK New Media,Sal...","https://siftery.com/company/adobe,https://sift..."
https://siftery.com/act-on,"Telestream,Texas Tech University,TripleLift,mG...","https://siftery.com/company/telestream,https:/..."
https://siftery.com/act-premium,"Innovative Systems,Sandler Training,KeeVac Ind...",https://siftery.com/company/innovative-systems...
https://siftery.com/activecampaign,"Renegade Empire,Earnworthy,Action 3 Media,Cupi...","https://siftery.com/company/renegade-empire,ht..."
https://siftery.com/adestra,"Vision2learn,Isabella Oliver,BBC History Magaz...","https://siftery.com/company/vision2learn,https..."


In [95]:
browser.quit()